In [1]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory

from config import OPENAI_API_KEY
from tools import create_rag_tool, create_google_search_tool, create_website_opener_tool

In [2]:
from langchain_teddynote import logging

logging.langsmith("LangChain_ReAct_Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain_ReAct_Agent


In [ ]:
tools = [
    create_rag_tool('detailed_graduation_rules.pdf', 'utf-8', 300, 100, 'detailed_graduation_rules_retriever', '서울대학교 산업공학과 졸업을 위한 졸업 요건에 대한 세부 사항을 물어보는 질문에 답할 때 사용'),
    create_google_search_tool(),
    create_website_opener_tool()
]

llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    api_key=OPENAI_API_KEY,
    temperature=0
)

prompt = hub.pull('hwchase17/react-chat')

agent = create_react_agent(llm, tools, prompt)

# ConversationBufferMemory vs ConversationSummaryMemory (추후, 결정하기)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools,
    memory=memory,
    # ReAct 과정 출력 여부
    verbose=True,
    # 출력 형식 오류 자동 처리 및 복구 여부
    handling_parsing_erros=True
)

C:\Users\dlaek\AppData\Local\Temp\ipykernel_29280\317418207.py:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [4]:
response1 = agent_executor.invoke({
    "input" : "안녕 내 이름은 다움이야. 서울대학교 산업공학과를 졸업하기 위해 필요한 학점은 얼마나 필요해? 전공, 교양 등 각기 영역별로 필요한 학점을 알려줘."
})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: detailed_graduation_rules_retriever
Action Input: 서울대학교 산업공학과 졸업 요건산업공학과학생을위한
교과목이수및졸업에관한학과내규
(2008∼202 학번통합본)
최종개정일
202 년3월31일
서울대학교공과대학
산업공학과
1이후 
1

산업공학과학생을위한
교과목이수및졸업에관한학과내규
(2008∼202 학번통합본)
최종개정일
202 년3월31일
서울대학교공과대학
산업공학과
1이후 
1

산업공학과학생을위한
교과목이수및졸업에관한학과내규
(2008∼202 학번통합본)
최종개정일
202 년3월31일
서울대학교공과대학
산업공학과
1이후 
1

‘전필학점’과‘학과개설전선학점’에 대한 이수요건을모두만족한 후‘주전공학점’ 이수학점
을만족해야 함.)
㈒ 학과 개설‘산업공학개론’ 교과목은소속학생의주전공으로 인정하지아니한다.
⑵ 학사과정 졸업(수료)을 위하여는 ‘대학원세미나 과목’ 1학점을 이수하여야 한다.
㈎ 대학원세미나 과목은매학년도1학기에만 개설됨.
㈏ 단, 아래의서울대학교학칙에 의하여, 대학원세미나 수업을1 · 2학년때 수강할 경우학사과
정 졸업학점에 포함되지않음을 유의하기바람. 따라서, 가능하면 대학원세미나 수업은3 · 4학년 때 수강하는 것이 좋다.

졸업을 위한 학점 요건은 다음과 같습니다:
1. 전공 필수 과목: 학과에서 정한 필수 과목을 모두 이수해야 합니다.
2. 전공 선택 과목: 학과에서 개설하는 선택 과목 중 일정 학점을 이수해야 합니다.
3. 교양 과목: 대학에서 요구하는 교양 과목 학점을 이수해야 합니다.
4. 대학원 세미나 과목: 적어도 1학점을 이수해야 하며, 이 과목은 매년 1학기에만 개설됩니다. 1학년과 2학년 때 수강할 경우 학사 과정 졸업 학점에 포함되지 않으므로 주의가 필요합니다.

상세한 학점 분배와 각 과목에 대한 자세한 정보는 학과의 최신 내규를 확인

In [5]:
response2 = agent_executor.invoke({
    "input" : "내 이름이 무엇인지 알아?"
})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: 네, 당신의 이름은 다움이라고 말씀하셨어요.

> Finished chain.


In [6]:
response3 = agent_executor.invoke({
    "input" : "내가 처음 물어본 질문이 무엇이었지?"
})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: 처음에 다움님이 물어보신 질문은 "서울대학교 산업공학과를 졸업하기 위해 필요한 학점은 얼마나 필요해? 전공, 교양 등 각기 영역별로 필요한 학점을 알려줘." 였습니다.

> Finished chain.


In [7]:
print(response1['output'])

서울대학교 산업공학과 졸업을 위해서는 전공 필수 과목, 전공 선택 과목, 교양 과목 등을 포함한 일정 학점을 이수해야 합니다. 또한, 대학원 세미나 과목을 최소 1학점 이수해야 하며, 이 과목은 매년 1학기에만 개설되므로 계획을 잘 세워야 합니다. 자세한 학점 요건과 과목 구성은 학과의 최신 내규를 참조하거나 학과 사무실에 문의하는 것이 좋습니다.


In [8]:
print(response2['output'])

네, 당신의 이름은 다움이라고 말씀하셨어요.


In [9]:
print(response3['output'])

처음에 다움님이 물어보신 질문은 "서울대학교 산업공학과를 졸업하기 위해 필요한 학점은 얼마나 필요해? 전공, 교양 등 각기 영역별로 필요한 학점을 알려줘." 였습니다.
